In [2]:
import matplotlib.pyplot as plt

In [6]:
# Read JSON
import json
json_file_path = '2021-11-19.json'

with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())

In [3]:
'''
Draws a football field chart 
param: ticker(string)
       db(dataframe)
return: bar chart(pyplot) with price tags
'''
def valuation_chart(ticker, db):
    ticker = ticker.upper()
    #values needed to plot footbaal field chart
    db['Range diff'] = db['52W Range High'] - db['52W Range Low']
    db['DCF ms=0.5'] = db['DCF Value (tv=10']*0.5 #lowest DCF value * lowest margin
    db['DCF Value diff'] = db['DCF Value (tv=15)'] - db['DCF ms=0.5']
    db['SP diff'] = db['SP Fair Value'] - db['SP Buy Value (ms=0.5)']
    
    row = db.loc[db['index'] == ticker].iloc[0]
    
    df = pd.DataFrame(columns=['low', 'diff', 'high'], index=['DCF Value', 'SP Value', 'Range 52W H/L'])
    df.iloc[0]['low'] = row['DCF ms=0.5'] #lowest possible buy value
    df.iloc[0]['diff'] = row['DCF Value diff'] #what I want to see
    df.iloc[0]['high'] = row['DCF Value (tv=15)'] #best fair value
    df.iloc[1]['low'] = row['SP Buy Value (ms=0.5)']
    df.iloc[1]['diff'] = row['SP diff']
    df.iloc[1]['high'] = row['SP Fair Value']
    df.iloc[2]['low'] = row['52W Range Low']
    df.iloc[2]['diff'] = row['Range diff']
    df.iloc[2]['high'] = row['52W Range High']
    
    price= row['Price']

    fig, ax=plt.subplots()
    df.plot(kind='bar', stacked=True, color=['w','burlywood','w'], legend=False, ax=ax)
    ax.axhline(y=price, color='r', linestyle='dotted')
    plt.title('Valuation summary, Ticker: ' + ticker)
    plt.ylabel('Share Price ($)')
    plt.xticks(rotation='horizontal')
    plt.ylim(min(df['low'])-30, max(df['high'])+100)

    # dynamic current price line labeling
    trans = transforms.blended_transform_factory(ax.get_yticklabels()[0].get_transform(), ax.transData)
    ax.text(0,price, "{:.0f}".format(price), color="red", transform=trans, ha="right", va="center")

    #del str
    labels_1 = []
    labels_2 = []
    for j in df.columns:
        for i in df.index:
            label = "$" + str(round(df.loc[i][j], 2))
            labels_1.append(label)
            labels_2.append(label)

    patches = ax.patches

    for i in range(3, len(labels_1)):
        labels_1[i] = ''
        #labels_1 = ['$63.23', '$33.53', '$51.06', '', '', '', '', '', '']
    for label, rect in zip(labels_1, patches):
        width = rect.get_width()
        if width > 0:
            x = rect.get_x()
            y = rect.get_y()
            height = rect.get_height()
            ax.text(x + width/2., y + height/1.1, label, ha='center', va='top')

    for i in range(len(labels_2)-3):
        labels_2[i] = ''
        #labels_2 = ['', '', '', '', '', '', '$78.88', '$44.7', '$134.8']
    for label, rect in zip(labels_2, patches):
        width = rect.get_width()
        if width > 0:
            x = rect.get_x()
            y = rect.get_y()
            height = rect.get_height()
            ax.text(x + width/2., y + height/14, label, ha='center', va='bottom')